In [13]:
import os
import json
import torch
import argparse
import open3d as o3d
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


cls_names = {"Pedestrian", "RoadBarrel"}


class ModelNetDataLoader(Dataset):
    def __init__(self, root, bag):
        super().__init__()
        self.root = root
        self.bag = bag
        self.bag_path = os.path.join(self.root, self.bag)
        self.jsons_path = os.path.join(self.bag_path, "label")
        self.pcds_path = os.path.join(self.bag_path, "lidar")

        # 点云和json文件目录检验
        pcd_list = sorted(os.listdir(self.pcds_path))
        json_list = sorted(os.listdir(self.jsons_path))
        if (len(pcd_list) != len(json_list)):
            for i in range(len(pcd_list)):
                pcd = pcd_list[i].rsplit('.', maxsplit=1)[0]
                jsn = json_list[i].rsplit('.', maxsplit=1)[0]
                assert pcd == jsn, ("%s: PCD files and JSON files do not match." % bag)

        # 加载点云和json文件目录
        self.dataset = [(os.path.join(self.pcds_path, pcd_list[i]), os.path.join(self.jsons_path, json_list[i])) for i in range(len(pcd_list))]
        print('The size of %s dataset is %d' % (self.bag, len(self.dataset)))

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        pcd_file = self.dataset[index][0]
        json_file = self.dataset[index][1]
        pcd = o3d.io.read_point_cloud(pcd_file)
        pnts = torch.tensor(np.asarray(pcd.points), dtype=torch.float32)
        objs = self.load_json(json_file)
        return pnts, objs

    def load_json(self, json_file):
        """ 加载json文件

        Args:
            json_file (_type_): json文件
        """
        with open(json_file, 'r') as f:
            content = f.read().strip()
        json_msg = json.loads(content)

        objs = []
        for obj in json_msg:
            label = obj["obj_type"]
            position = obj["psr"]["position"]
            scale = obj["psr"]["scale"]
            objs.append([label,
                         torch.tensor([position["x"], position["y"], position["z"]], dtype=torch.float32),
                         torch.tensor([scale["x"], scale["y"], scale["z"]], dtype=torch.float32)])
            return objs


def parse_args():
    parser = argparse.ArgumentParser('SphercialRobotParser')
    parser.add_argument('--data_path', default="/home/luoteng/PointNet2/data/sphercial_robot_data/", type=str, required=False, help='Experimental dataset root')
    return parser.parse_args()




In [36]:
data_root = "/home/luoteng/PointNet2/data/sphercial_robot_data/"
bags = os.listdir(data_root)

In [37]:
bag = bags[0]
dataset = ModelNetDataLoader(data_root, bag)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=10)

The size of rotun_0601_p0 dataset is 299


In [38]:
pc, objs = dataset[0]
obj = objs[0]

In [39]:
pc, obj

(tensor([[ 6.0610,  1.5842, -0.1278],
         [ 5.9847,  1.4064, -0.1320],
         [ 5.9432,  1.6804, -0.1332],
         ...,
         [-8.8913, -3.8161, -0.6051],
         [-7.9448, -3.0163, -0.6528],
         [-8.9624, -3.4133, -0.6172]]),
 ['Pedestrian',
  tensor([-2.8255, -4.2680,  0.2692]),
  tensor([0.5679, 0.4433, 1.7275])])

In [40]:
label = obj[0]
center = obj[1]
scale = obj[2]
center, scale

(tensor([-2.8255, -4.2680,  0.2692]), tensor([0.5679, 0.4433, 1.7275]))

In [41]:
max_pt = center + scale / 2 
min_pt = center - scale / 2 

In [72]:
result= (min_pt<pc)  & (pc<max_pt)
mask = torch.all(result, dim=1)
obj_pc = pc[mask]